# C++: Why do we need more?

* Sometimes simple is enough!
* `export "C"` allows C++ backend
* C++ API can have: overloading, classes, memory management, etc...
* We could manually translate everything using C API

### Solution:

C++ binding tools!

This is our C++ example:

In [ ]:
import sys

if sys.platform.startswith("darwin"):
    %set_env CPATH=/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include
    %set_env LIBRARY_PATH=/Library/Developer/CommandLineTools/SDKs/MacOSX.sdk/usr/lib

In [ ]:
%%writefile SimpleClass.hpp
#pragma once

class Simple {
    int x;
    
public:

    Simple(int x): x(x) {}
    
    int get() const {
        return x;
    }
    
};

[![SWIG](swigfull.png)](swig.org)

* SWIG: Produces "automatic" bindings
* Works with many output languages
* Has supporting module built into CMake
* Very mature

Downsides:

* Can be all or nothing
* Hard to customize
* Customizations tend to be language specific
* Slow development

In [ ]:
%%writefile swigexample.i

%module swigexample
%{
/* Includes the header in the wrapper code */
#include "SimpleClass.hpp"
%}
 
/* Parse the header file to generate wrappers */
%include "SimpleClass.hpp"

In [ ]:
!swig -c++ -python -py3 swigexample.i

In [ ]:
%%writefile setup.py

from setuptools import setup, Extension
import sys

module1 = Extension('_swigexample',
                    sources=['swigexample_wrap.cxx'],
                    language='cxx',
                    extra_compile_args=['-std=c++11'],
                   )

setup(name='swigexample', ext_modules=[module1])

In [ ]:
!python3 setup.py build_ext --inplace

In [ ]:
import swigexample

x = swigexample.Simple(2)
x.get()